In [1]:
import os
import pandapower as pp
import polars as pl
from polars import col as c

from pipelines.dig_a_plan_d_model import DigAPlan
from data_display.grid_plotting import plot_grid_from_pandapower
from data_display.output_processing import compare_dig_a_plan_with_pandapower
from data_connector import change_schema_to_dig_a_plan_schema, duckdb_to_changes_schema
from twindigrid_changes.schema import ChangesSchema
from networkx_function import generate_nx_edge, get_connected_edges_data, generate_bfs_tree_with_edge_data, get_all_edge_data
import networkx as nx

import matplotlib.pyplot as plt

os.chdir(os.getcwd().replace("/src", ""))
os.environ['GRB_LICENSE_FILE'] = os.environ["HOME"] + "/gurobi_license/gurobi.lic"


In [2]:
change_schema: ChangesSchema = duckdb_to_changes_schema(file_path=".cache/input_data/boisy_grid.db")
# 2) build the *original* schema once

base_grid_data = change_schema_to_dig_a_plan_schema(change_schema = change_schema)

nx_graph = nx.Graph()
_ = base_grid_data["edge_data"].with_columns(
    pl.struct(pl.all()).pipe(generate_nx_edge, nx_graph=nx_graph)
)

max_n_trafo = 1.08
edge_id  = get_connected_edges_data(nx_graph = nx_graph).filter(c("graph_id")!= 0)["edge_id"].to_list()
node_id = get_connected_edges_data(nx_graph = nx_graph).filter(c("graph_id")!= 0)\
    .unpivot(on=["u_of_edge", "v_of_edge"])["value"].to_list()
base_grid_data["edge_data"] = base_grid_data["edge_data"].filter(~c("edge_id").is_in(edge_id))
base_grid_data["node_data"] = base_grid_data["node_data"].filter(~c("node_id").is_in(node_id))
base_grid_data["edge_data"] = base_grid_data["edge_data"].with_columns(
    pl.when(c("b_pu") < 1e-6).then(pl.lit(0)).otherwise(c("b_pu")).alias("b_pu"),
    pl.when(c("n_transfo") > max_n_trafo).then(pl.lit(max_n_trafo)).otherwise(c("n_transfo")).alias("n_transfo")
    )


Read and validate tables from boisy_grid.db file:   0%|                                                                        | 0/28 [00:00<?, ?it/s]

Read and validate tables from boisy_grid.db file: 100%|██████████████████████████████████████████████████████████████| 28/28 [00:00<00:00, 324.20it/s]


In [3]:

dig_a_plan: DigAPlan = DigAPlan(
    verbose= False, big_m = 1e4, penalty_cost = 1e1, convergence_threshold = 1e-5
)

dig_a_plan.add_grid_data(**base_grid_data)
dig_a_plan.solve_models_pipeline(max_iters = 200)

Master obj: 1.9E-02, Slave obj: 1.9E-02 and Gap: -1.0E-14:   0%|          | 0/200 [00:01<?, ?it/s]


In [4]:
dig_a_plan.edge_data.filter(c("type")== "switch")["eq_fk", "normal_open"].with_columns(
    (~c("eq_fk").replace_strict(dig_a_plan.extract_switch_status(), default=None)).alias("open"),
).filter(c("open") != c("normal_open"))

eq_fk,normal_open,open
str,bool,bool
"""d9255bb0-767a-5eec-81ff-0a3df1…",true,false
"""922efaff-4656-579c-8b53-e525e2…",true,false
"""0e05522f-cbe2-5b34-ae55-9a9204…",true,false
"""9bacd1d0-7b6f-591e-b3c3-369f5b…",true,false
"""d6a252aa-405e-5c86-958e-e4cdc5…",true,false
…,…,…
"""c74c0a4c-9e4f-5776-9ef9-eccd2e…",true,false
"""eb3a347a-2cdf-57be-b1ba-91df00…",false,true
"""35eb3bc4-8b29-526d-a962-d81238…",true,false


In [5]:
dig_a_plan.extract_switch_status()

{'8dbec791-0228-5627-a843-2bf35fd8e144': True,
 '69a27e56-783b-5035-b29c-d5504a1811c9': True,
 '19da717d-66b8-5214-9e55-10ec5308f856': True,
 'bd77cbbd-2689-5ae3-a6c9-62f159a1732b': True,
 '3701ef23-c779-5179-b360-c38198f6b0ea': True,
 'e9389c06-4a2b-558f-b753-f8019f1141a4': True,
 'd9255bb0-767a-5eec-81ff-0a3df19f8278': True,
 '7a385534-a661-5921-bb52-4f6b57348162': True,
 '2a5ef031-c72a-5642-8e59-1ba303888b1f': True,
 '846effff-caff-58dc-97a4-a8189bc9f609': True,
 '41998f82-5789-51e1-9b87-45c74b1597aa': True,
 '16f0053f-9a59-5141-958c-3e3820dabc8c': True,
 'f3fb98b2-caeb-5d78-a1d2-7d0def73c558': True,
 '446a60c7-0913-5ec9-a7e7-85987168eb65': True,
 '922efaff-4656-579c-8b53-e525e229132a': True,
 'ed143c3e-82cc-5976-aa7b-e81e89e8d15f': False,
 '11d3d73d-c612-54d2-b178-856fcfdf5fc2': True,
 '625476db-82e8-558e-8507-6a2aae3b765a': True,
 '0e05522f-cbe2-5b34-ae55-9a92049d4646': True,
 '58c8d26a-a277-596a-bd26-f0326b7626c4': True,
 '9bacd1d0-7b6f-591e-b3c3-369f5b5e2739': True,
 'b8273518-f